In [10]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline

# Use a white background for matplotlib figures
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [11]:
unique_mutations = ['30N','32I','46I','46L','47A','47V','48A','48S','48T','48Q','48L','48M','48V','50L','50V','54A',
                   '54T','54S','54L','54M','54V','76V','82A','82C','82F','82L','82M','82T','82S','84A',
                   '84C','84V','88S','88T','88G','90M']
len(unique_mutations)

36

In [12]:
df = pd.read_csv("datawithnonulls.csv")
raw = df.copy()

In [13]:
raw

,Mutation 1,Mutation 2,Mutation 3,Mutation 4,Mutation 5,Mutation 6,Mutation 7,NFV,SQV,IDV,FPV,ATV,LPV,TPV,DRV
0,90M,NaN,NaN,NaN,NaN,NaN,NaN,7.3,3.2,3.1,1.3,3.1,1.6,1.2,0.9
1,30N,NaN,NaN,NaN,NaN,NaN,NaN,40.0,1.3,1.3,0.8,2.5,1.0,1.2,1.0
2,46I,90M,NaN,NaN,NaN,NaN,NaN,22.0,4.2,9.1,3.2,4.9,2.7,0.9,0.8
3,46I,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.9,4.4,2.1,2.2,2.7,1.1,0.8
4,54V,82A,90M,NaN,NaN,NaN,NaN,57.0,27.0,24.0,3.6,36.0,39.0,2.4,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,32I,46L,47V,54M,90M,NaN,NaN,44.0,18.0,43.0,56.0,46.0,70.0,9.5,NaN
282,30N,46L,54L,NaN,NaN,NaN,NaN,133.0,21.0,1.4,8.2,35.0,4.6,0.2,1.7
283,46L,48M,54M,82A,84V,NaN,NaN,7.2,79.0,8.2,47.0,24.0,41.0,4.5,13.0
284,32I,46I,47V,50L,82T,90M,NaN,8.3,1.3,13.0,2.2,161.0,8.0,4.5,0.7


In [15]:
rawlist = raw.values.tolist()
rawlist

[['90M', nan, nan, nan, nan, nan, nan, 7.3, 3.2, 3.1, 1.3, 3.1, 1.6, 1.2, 0.9],
 ['30N',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  40.0,
  1.3,
  1.3,
  0.8,
  2.5,
  1.0,
  1.2,
  1.0],
 ['46I',
  '90M',
  nan,
  nan,
  nan,
  nan,
  nan,
  22.0,
  4.2,
  9.1,
  3.2,
  4.9,
  2.7,
  0.9,
  0.8],
 ['46I',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  11.0,
  0.9,
  4.4,
  2.1,
  2.2,
  2.7,
  1.1,
  0.8],
 ['54V',
  '82A',
  '90M',
  nan,
  nan,
  nan,
  nan,
  57.0,
  27.0,
  24.0,
  3.6,
  36.0,
  39.0,
  2.4,
  1.6],
 ['54V',
  '82A',
  nan,
  nan,
  nan,
  nan,
  nan,
  9.4,
  1.4,
  8.0,
  1.5,
  2.3,
  11.0,
  1.6,
  0.8],
 ['46L', nan, nan, nan, nan, nan, nan, 8.2, 1.5, 2.6, 1.1, 2.4, 1.6, nan, nan],
 ['46L',
  '54V',
  '82A',
  nan,
  nan,
  nan,
  nan,
  39.0,
  6.2,
  19.0,
  4.6,
  31.0,
  37.0,
  1.1,
  1.7],
 ['46I',
  '54V',
  '82A',
  nan,
  nan,
  nan,
  nan,
  16.0,
  1.6,
  15.0,
  2.5,
  7.5,
  27.0,
  1.6,
  0.8],
 ['46I',
  '84V',
  '90M',
  nan,
  nan,
  nan,
  na

In [17]:


matrix = []

for i in range(len(rawlist)):
    newl = []
    for a in range(36):
        newl.append(0)
    for j in range(7):
        if not pd.isnull(rawlist[i][j]):
            ind = unique_mutations.index(rawlist[i][j])
            newl[ind] = 1
    matrix.append(newl)
len(matrix)

matrix

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [18]:
mappedmatrix = pd.DataFrame(columns = unique_mutations)
mappedmatrix

for i in range(len(matrix)):
    mappedmatrix.loc[i] = matrix[i]
mappedmatrix.to_csv('MappedMatrix_updated.csv',index = 0)

In [19]:
mapped = pd.read_csv('MappedMatrix_updated.csv')
mapped

,30N,32I,46I,46L,47A,47V,48A,48S,48T,48Q,...,82M,82T,82S,84A,84C,84V,88S,88T,88G,90M
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
282,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
284,0,1,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [9]:
import torch

nfvinputs = mapped.to_numpy()
nfvinputs


array([[0, 0, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [10]:
inputs = torch.from_numpy(nfvinputs)

outputs = nfvoutputs.to_numpy()
targets = torch.from_numpy(outputs)
targets.size()

torch.Size([284, 1])

In [11]:
input_size = inputs.shape[-1]
hidden_size = 16

input_size

36

In [12]:
layer1 = nn.Linear(input_size, hidden_size)

In [13]:
inputs.shape

torch.Size([284, 36])

In [14]:
print(inputs)
print(targets)

tensor([[0, 0, 0,  ..., 0, 0, 1],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 1],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 1],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[7.3000e+00],
        [4.0000e+01],
        [2.2000e+01],
        [1.1000e+01],
        [5.7000e+01],
        [9.4000e+00],
        [8.2000e+00],
        [3.9000e+01],
        [1.6000e+01],
        [4.8000e+01],
        [4.0000e+01],
        [1.9000e+01],
        [8.6000e+01],
        [4.9000e+01],
        [2.1000e+00],
        [7.7000e+00],
        [1.1000e+01],
        [1.4400e+02],
        [3.4000e+01],
        [7.4000e+01],
        [3.0000e-01],
        [2.5000e+00],
        [1.8000e+01],
        [6.1000e+00],
        [1.8000e+01],
        [2.5000e+01],
        [6.1000e+00],
        [1.3000e+01],
        [1.5000e+01],
        [2.9000e+01],
        [1.1000e+02],
        [5.0000e+01],
        [6.7000e+01],
        [2.3700e+02],
        [2.5000e+01],
        [8.9

In [15]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs.float(), targets.float())
train_ds[0:3]

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]),
 tensor([[ 7.3000],
         [40.0000],
         [22.0000]]))

In [16]:
targets.size()

torch.Size([284, 1])

In [17]:
inputs.size()

torch.Size([284, 36])

In [18]:
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [19]:
import torch.nn as nn

model = nn.Linear(36,1)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.1506, -0.0306, -0.1562,  0.0071, -0.1550,  0.0180,  0.1291, -0.0267,
         -0.0746,  0.0009,  0.1298, -0.0070, -0.1296,  0.0929,  0.0881, -0.1455,
         -0.1296, -0.0629, -0.0559,  0.0448, -0.0479, -0.0940,  0.0746, -0.0012,
          0.1386,  0.1253,  0.0364,  0.0589,  0.0245, -0.1280,  0.0390, -0.0827,
         -0.0355,  0.0087, -0.0254,  0.1035]], requires_grad=True)
Parameter containing:
tensor([0.0464], requires_grad=True)


In [20]:
preds = model(inputs.float())
preds

tensor([[ 1.4998e-01],
        [-1.0415e-01],
        [-6.2602e-03],
        [-1.0980e-01],
        [ 1.7668e-01],
        [ 7.3137e-02],
        [ 5.3522e-02],
        [ 8.0219e-02],
        [-8.3103e-02],
        [-8.8982e-02],
        [ 1.8376e-01],
        [ 6.7258e-02],
        [-1.3692e-01],
        [ 2.0436e-02],
        [ 1.2108e-01],
        [ 1.0916e-02],
        [-1.7714e-01],
        [ 1.9316e-02],
        [ 1.0204e-01],
        [-6.1625e-04],
        [ 1.3933e-01],
        [-3.6280e-02],
        [-1.4532e-01],
        [ 1.2816e-01],
        [-1.5009e-03],
        [-2.6039e-01],
        [ 2.2462e-01],
        [-1.9252e-01],
        [ 1.5706e-01],
        [-5.6429e-02],
        [ 8.4400e-02],
        [ 1.1392e-02],
        [-3.5160e-02],
        [-1.1897e-01],
        [ 3.7787e-02],
        [ 2.6397e-02],
        [ 1.0053e-01],
        [-2.5031e-03],
        [-1.6582e-01],
        [-1.3806e-01],
        [-2.4046e-01],
        [-7.3603e-02],
        [ 2.4064e-01],
        [-6

In [21]:
import torch.nn.functional as F
loss_fn = F.mse_loss
loss = loss_fn(model(inputs.float()), targets.float())
print(loss)

tensor(24479.8438, grad_fn=<MseLossBackward0>)


In [23]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [24]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [26]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 84127.7578
Epoch [20/100], Loss: 2098.2881
Epoch [30/100], Loss: 2730.0398
Epoch [40/100], Loss: 41003.1055
Epoch [50/100], Loss: 10786.3760
Epoch [60/100], Loss: 931.8007
Epoch [70/100], Loss: 488.8400
Epoch [80/100], Loss: 2741.5977
Epoch [90/100], Loss: 216.3271
Epoch [100/100], Loss: 83519.1016
